# Credit Card Fraud Detection 


In [1]:
#importing the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [2]:
#importing files
Credit_card_data=pd.read_csv("creditcard.csv")
Credit_card_data.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [3]:
#dataset information
Credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
#checking wheather dataset consist Null values
Credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
#Distribution in Legit and Fraud Transaction
Credit_card_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

The dataset are highly unbalanced with dominant legit transaction

In [6]:
#Separating dataset for analysis
legit=Credit_card_data[Credit_card_data.Class == 0]
fraud=Credit_card_data[Credit_card_data.Class == 1]

In [7]:
#Statistical measures of ddataset
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [8]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [9]:
#Comparing the values for both transaction
Credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


 Under sampling the value of legit transaction to match same number of transaction with fraud transaction (legit-->492)

In [10]:
legit_sample=legit.sample(n=492)

In [11]:
legit_sample.shape

(492, 31)

Concatinating two dataframes to create new dataset

In [12]:
new_dataset=pd.concat([legit_sample,fraud],axis=0)

In [13]:
new_dataset.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
178365,123600.0,-1.429759,1.898700,-0.732820,-0.908957,0.932209,-0.836241,1.485268,-0.704606,1.394518,...,0.003110,1.298003,-0.232587,0.804677,-0.417219,-0.277780,0.997069,0.334322,38.51,0
278265,168127.0,2.140718,-1.681938,-0.937464,-1.717303,-1.134333,0.152518,-1.339267,0.055444,-1.286162,...,-0.079042,0.096655,0.186660,0.273146,-0.281908,-0.207526,0.008572,-0.040953,79.00,0
195620,131146.0,0.009018,0.833921,0.452929,-0.491425,0.182055,-1.285924,0.974999,-0.177979,0.007282,...,-0.233952,-0.515797,0.133832,0.378455,-0.577827,0.119618,0.247403,0.099800,3.57,0


In [14]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96292.461382,-0.086906,-0.005730,-0.052011,-0.069299,-0.001198,-0.061990,-0.056780,0.019441,0.038207,...,-0.021089,-0.042282,0.031577,-0.060301,0.087921,0.000068,0.011166,0.029735,-0.001433,79.025711
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [15]:
#Splitting dataset into Features and Targetabs 
X=new_dataset.drop(columns='Class',axis=1)
Y=new_dataset['Class']

In [16]:
print(Y)

178365    0
278265    0
195620    0
222169    0
220620    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [17]:
#Assigning training and testing data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)

Using Maching Learning models(LogisticRegression,SVM,DecisionTreeClassifier)

In [18]:
#Using LogisticRegression Model
model=LogisticRegression(max_iter=100000)


In [19]:
#Training thhe data

model.fit(X_train,Y_train)


LogisticRegression(max_iter=100000)

Model Evaluation

In [20]:
#Checking for acccuracy in training dataset
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [21]:
print('Accuracy score for train data : ',training_data_accuracy)

Accuracy score for train data :  0.9440914866581956


In [22]:
#Checking for acccuracy in testing dataset
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [23]:
print('Accuracy score for test data : ',testing_data_accuracy)

Accuracy score for test data :  0.9543147208121827


In [24]:
#Using support vecter machine model
model2=SVC(kernel='linear',degree=5)

In [25]:
model2.fit(X_train,Y_train)

SVC(degree=5, kernel='linear')

In [26]:
#Checking for acccuracy in training dataset
X_train_prediction=model2.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [27]:
print('Accuracy score for train data : ',training_data_accuracy)

Accuracy score for train data :  0.8843710292249047


In [28]:
X_test_prediction=model2.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [29]:
print('Accuracy score for test data : ',testing_data_accuracy)

Accuracy score for test data :  0.9086294416243654


In [30]:
#Using DecisionTreeClassifier model
model3=DecisionTreeClassifier(max_depth=4)

In [31]:
model3.fit(X_train,Y_train)

DecisionTreeClassifier(max_depth=4)

In [32]:
#Checking for acccuracy in training dataset
X_train_prediction=model3.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [33]:
print('Accuracy score for train data : ',training_data_accuracy)

Accuracy score for train data :  0.9593392630241423


In [34]:
X_test_prediction=model3.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [35]:
print('Accuracy score for test data : ',testing_data_accuracy)

Accuracy score for test data :  0.9238578680203046


Using Esemble model

In [43]:
esm=[("lg1",LogisticRegression(max_iter=10000)),("sv1",SVC(kernel='linear',degree=5)),("dt1",DecisionTreeClassifier(max_depth=5))]

In [44]:
vc=VotingClassifier(esm)

In [45]:
vc.fit(X_train,Y_train)

VotingClassifier(estimators=[('lg1', LogisticRegression(max_iter=10000)),
                             ('sv1', SVC(degree=5, kernel='linear')),
                             ('dt1', DecisionTreeClassifier(max_depth=5))])

In [46]:
X_train_prediction=vc.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [47]:
print('Accuracy score for train data : ',training_data_accuracy)

Accuracy score for train data :  0.9491740787801779


In [48]:
X_test_prediction=model3.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [49]:
print('Accuracy score for test data : ',testing_data_accuracy)

Accuracy score for test data :  0.9238578680203046


Conclusion: Model providing accuracy above 90 % is considered good model for implementation